<a href="https://colab.research.google.com/github/ImonHBnLT/Colab/blob/main/prog_linear_pesq_operacional_carga_veiculo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 20.8 MB/s eta 0:00:00


In [ ]:
#Trabalho 1-	Exercício  1:
#a)

# Cria função para ler dados do txt
def ler_dados(arquivo):
  with open(arquivo, 'r') as f:
    linhas = f.readlines()
  Pmax = int(linhas[0].strip()) # Peso máximo
  n = int(linhas[1].strip()) # Itens disponiveis
  P = [] # Peso do produto
  V = [] # Valor do produto
  Q = [] # Quantidade do produto

  for i in range(2, 2+n): # Leitura das informações dos itens
    p, v, q = map(int, linhas[i].split())
    P.append(p)
    V.append(v)
    Q.append(q)

  return n, Pmax, P, V, Q

arquivo='dados.txt'
n, Pmax, P, V, Q = ler_dados(arquivo)

In [ ]:
#Trabalho 1- Exercício 1:
#a)

import gurobipy as gp

CargaVeiculo = gp.Model() # Cria um modelo chamado CargaVeiculo

# variáveis de decisão:
x = CargaVeiculo.addVars(n, vtype = gp.GRB.INTEGER) # cria n variáveis inteiras

# função objetivo:
CargaVeiculo.setObjective(sum(V[i] * x[i] for i in range(n)), sense=gp.GRB.MAXIMIZE)

# Restrições
  # Restrição de peso máximo do veículo
r1 = CargaVeiculo.addConstr(
    sum(P[i] * x[i] for i in range(n)) <= Pmax
)
  # Restrição de quantidade máxima de cada item
r2 = CargaVeiculo.addConstrs(
    (x[i] <= Q[i] for i in range(n))
)

# Suprimindo console output
CargaVeiculo.setParam('OutputFlag', 0)

# Resolvendo o modelo
CargaVeiculo.optimize()

# Verificando status do modelo
print("Status do modelo:  {}".format(CargaVeiculo.status))
if CargaVeiculo.status==2:
  pesotot = 0
  print("Valor ótimo = {:.0f}".format(CargaVeiculo.objVal))
  for i in range(n):
    if round(x[i].x) > 0:
      print(" Item {} = {:.0f}".format(i+1, x[i].x))
      pesotot = pesotot + x[i].x * P[i]
  print("Peso total ocupado: {:.0f}".format(pesotot))
  print("Solução ótima encontrada!")
else:
  print("Solução ótima não encontrada!")

Status do modelo:  2
Valor ótimo = 2099
 Item 6 = 5
 Item 50 = 11
 Item 84 = 17
 Item 97 = 17
Peso total ocupado: 50
Solução ótima encontrada!


In [ ]:
#Trabalho 1-	Exercício  1:
#b)

# Cria função para ler dados do txt
def ler_dados(arquivo):
    with open(arquivo, 'r') as f:
        linhas = f.readlines()
    Pmax = int(linhas[0].strip())  # Peso máximo
    n = int(linhas[1].strip())  # Número de itens
    P = []  # Peso do produto
    V = []  # Valor do produto
    Q = []  # Quantidade do produto

    for i in range(2, 2 + n):  # Leitura das informações dos itens
        p, v, q = map(int, linhas[i].split())
        P.append(p)
        V.append(v)
        Q.append(q)

    Incomp = [] # Leitura dos itens incompatíveis
    for linha in linhas[2 + n:]:
        p1, p2 = map(int, linha.split())
        Incomp.append((p1, p2))

    return Pmax, n, P, V, Q, Incomp

arquivo = 'dados.txt'
Pmax, n, P, V, Q, Incomp = ler_dados(arquivo)

In [ ]:
import gurobipy as gp

CargaVeiculo = gp.Model()  # Cria um modelo chamado CargaVeiculo

# Variáveis de decisão:
x = CargaVeiculo.addVars(n, vtype=gp.GRB.INTEGER)  # Cria n variáveis inteiras
y = CargaVeiculo.addVars(n, vtype=gp.GRB.BINARY)  # Cria n variáveis binárias

# Função objetivo:
CargaVeiculo.setObjective(sum(V[i] * x[i] for i in range(n)), sense=gp.GRB.MAXIMIZE)

# Restrições
  # Restrição de peso máximo do veículo
r1 = CargaVeiculo.addConstr(
    sum(P[i] * x[i] for i in range(n)) <= Pmax
)
  # Restrição de quantidade máxima de cada item
r2 = CargaVeiculo.addConstrs(
    (x[i] <= Q[i] for i in range(n))
)
  # Restrição binária de adição do item, considerando as quantidades dos itens
r3 = CargaVeiculo.addConstrs(
    (x[i] <= y[i] * Q[i] for i in range(n))
)
  # Restrição de incompatibilidade de itens
r4 = CargaVeiculo.addConstrs(
    (y[i-1] + y[j-1] <= 1 for (i, j) in Incomp)
)

# Suprimindo console output
CargaVeiculo.setParam('OutputFlag', 0)

# Resolvendo o modelo
CargaVeiculo.optimize()

# Verificando status do modelo
print("Status do modelo:  {}".format(CargaVeiculo.status))
if CargaVeiculo.status == 2:
    print("Valor ótimo = {:.0f}".format(CargaVeiculo.objVal))
    for i in range(n):
      pesotot = 0
      if round(x[i].x) > 0:
          print(" Item {} = {:.0f}".format(i + 1, x[i].x))
          pesotot = pesotot + x[i].x * P[i]
    print("Peso total ocupado: {:.0f}".format(pesotot))
    print("Solução ótima encontrada!")
else:
    print("Solução ótima não encontrada!")

Status do modelo:  2
Valor ótimo = 1997
 Item 6 = 6
 Item 69 = 1
 Item 84 = 17
 Item 96 = 9
 Item 97 = 17
Peso total ocupado: 0
Solução ótima encontrada!
